In [1]:
import pandas as pd

# 파일 경로들 정리
file_paths = {
    "가로등": r"데이터/동대문구 가로등 위치 정보.csv",
    "노인복지시설": r"데이터/동대문구 사회복지시설(노인여가복지시설).csv",
    "노인일자리기관": r"데이터/동대문구 사회복지시설(노인일자리지원기관).csv",
    "제가노인복지시설": r"데이터/동대문구 사회복지시설(재가노인복지시설).csv",
    "cctv": r"데이터/동대문구_CCTV.csv",
    "교차로": r"데이터/동대문구_교차로_위치찐.csv",
    "버스정류소": r"데이터/동대문구_버스정류소.csv",
    "병의원": r"데이터/동대문구_병의원.csv",
    "전통시장": r"데이터/동대문구_전통시장.csv",
    "지하철역": r"데이터/동대문구_지하철역.csv",
    "횡단보도": r"데이터/동대문구_횡단보도_중심좌표.csv",
    "led_횡단보도": r"/Users/nogeon-u/Desktop/gong/동대문구 공모전/시각화용 데이터/서울특별시 동대문구_바닥신호등 현황_20240318.csv",
    "보행등": r"데이터/서울특별시_보행등 위치좌표 현황_20221223.csv",
    "차량출입구": r"데이터/동대문구_차량출입구.csv",
    "단속카메라": r"데이터/서울특별시_동대문구_무인교통단속카메라_20240604.csv"


}

# 안전한 CSV 로딩 함수 (인코딩 자동 처리)
def load_csv_safely(path):
    for enc in ['utf-8', 'cp949', 'euc-kr']:
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"인코딩 문제로 파일을 불러올 수 없습니다: {path}")

# 모든 데이터프레임 불러오기
dataframes = {name: load_csv_safely(path) for name, path in file_paths.items()}

# 각각 개별 변수로도 저장 가능 (선택사항)
가로등 = dataframes["가로등"]
노인복지시설 = dataframes["노인복지시설"]
노인일자리기관 = dataframes["노인일자리기관"]
제가노인복지시설 = dataframes["제가노인복지시설"]
cctv = dataframes["cctv"]
교차로 = dataframes["교차로"]
버스정류소 = dataframes["버스정류소"]
병의원 = dataframes["병의원"]
전통시장 = dataframes["전통시장"]
지하철역 = dataframes["지하철역"]
횡단보도 = dataframes["횡단보도"]
led_횡단보도 = dataframes["led_횡단보도"]
보행등 = dataframes["보행등"]
차량출입구 = dataframes["차량출입구"]
단속카메라 = dataframes["단속카메라"]


# 확인용 출력
for name, df in dataframes.items():
    print(f"{name}: {df.shape} rows")
    print(df.columns.tolist())
    print()


FileNotFoundError: [Errno 2] No such file or directory: '데이터/동대문구 가로등 위치 정보.csv'

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box, Point
import matplotlib.pyplot as plt
import folium
from branca.colormap import LinearColormap

# 동대문구 shp 파일 불러오기
shp_path = "데이터/sig_20230729/sig.shp"
gdf = gpd.read_file(shp_path)
dongdaemun = gdf[gdf["SIG_ENG_NM"] == "Dongdaemun-gu"]
if dongdaemun.crs is None:
    dongdaemun = dongdaemun.set_crs(epsg=5179)
dongdaemun = dongdaemun.to_crs(epsg=4326)  # WGS84로 변환

# 동대문구 경계 좌표 가져오기
bounds = dongdaemun.total_bounds  # [minx, miny, maxx, maxy]
dongdaemun_bounds = {
    'min_lon': bounds[0],  # 최소 경도
    'max_lon': bounds[2],  # 최대 경도
    'min_lat': bounds[1],  # 최소 위도
    'max_lat': bounds[3]   # 최대 위도
}

# 30x30 격자 생성
def create_grid(bounds, n_rows=30, n_cols=30):
    lon_step = (bounds['max_lon'] - bounds['min_lon']) / n_cols
    lat_step = (bounds['max_lat'] - bounds['min_lat']) / n_rows
    
    grid_cells = []
    for i in range(n_rows):
        for j in range(n_cols):
            min_lon = bounds['min_lon'] + j * lon_step
            max_lon = min_lon + lon_step
            min_lat = bounds['min_lat'] + i * lat_step
            max_lat = min_lat + lat_step
            
            cell = box(min_lon, min_lat, max_lon, max_lat)
            grid_cells.append({
                'cell_id': f'{i}_{j}',
                'geometry': cell,
                'row': i,
                'col': j
            })
    
    return gpd.GeoDataFrame(grid_cells, crs="EPSG:4326")

# 시설 데이터를 격자에 매핑하는 함수
def count_facilities_in_grid(facilities_df, grid_gdf, lon_col, lat_col):
    counts = np.zeros(len(grid_gdf))
    
    # 필요한 컬럼이 있는지 확인
    if lon_col not in facilities_df.columns or lat_col not in facilities_df.columns:
        print(f"경고: {lon_col} 또는 {lat_col} 컬럼이 데이터프레임에 없습니다.")
        print(f"사용 가능한 컬럼: {facilities_df.columns.tolist()}")
        return counts
    
    # 시설 데이터를 GeoDataFrame으로 변환
    facilities_gdf = gpd.GeoDataFrame(
        facilities_df,
        geometry=[Point(xy) for xy in zip(facilities_df[lon_col], facilities_df[lat_col])],
        crs="EPSG:4326"
    )
    
    for idx, facility in facilities_gdf.iterrows():
        for i, grid_cell in grid_gdf.iterrows():
            if grid_cell.geometry.contains(facility.geometry):
                counts[i] += 1
                break
    
    return counts

# 파일 경로들 정리
file_paths = {
    "가로등": "데이터/동대문구 가로등 위치 정보.csv",
    "노인복지시설": "데이터/동대문구 사회복지시설(노인여가복지시설).csv",
    "노인일자리기관": "데이터/동대문구 사회복지시설(노인일자리지원기관).csv",
    "제가노인복지시설": "데이터/동대문구 사회복지시설(재가노인복지시설).csv",
    "cctv": "데이터/동대문구_CCTV.csv",
    "교차로": "데이터/동대문구_교차로_위치찐.csv",
    "버스정류소": "데이터/동대문구_버스정류소.csv",
    "병의원": "데이터/동대문구_병의원.csv",
    "전통시장": "데이터/동대문구_전통시장.csv",
    "지하철역": "데이터/동대문구_지하철역.csv",
    "횡단보도": "데이터/동대문구_횡단보도_중심좌표.csv",
    "led_횡단보도": "/Users/nogeon-u/Desktop/gong/동대문구 공모전/시각화용 데이터/서울특별시 동대문구_바닥신호등 현황_20240318.csv",
    "보행등": "데이터/서울특별시_보행등 위치좌표 현황_20221223.csv",
    "차량출입구": "데이터/동대문구_차량출입구.csv",
    "단속카메라": "데이터/서울특별시_동대문구_무인교통단속카메라_20240604.csv"
}

# 각 파일의 위도/경도 컬럼명 매핑
coord_columns = {
    "가로등": {"lat": "위도", "lon": "경도"},
    "노인복지시설": {"lat": "위도", "lon": "경도"},
    "노인일자리기관": {"lat": "위도", "lon": "경도"},
    "제가노인복지시설": {"lat": "위도", "lon": "경도"},
    "cctv": {"lat": "LA", "lon": "LO"},
    "교차로": {"lat": "위도", "lon": "경도"},
    "버스정류소": {"lat": "Y좌표", "lon": "X좌표"},
    "병의원": {"lat": "위도", "lon": "경도"},
    "전통시장": {"lat": "위도", "lon": "경도"},
    "지하철역": {"lat": "위도", "lon": "경도"},
    "횡단보도": {"lat": "위도", "lon": "경도"},
    "led_횡단보도": {"lat": "위도", "lon": "경도"},
    "보행등": {"lat": "위도", "lon": "경도"},
    "차량출입구": {"lat": "위도", "lon": "경도"},
    "단속카메라": {"lat": "위도", "lon": "경도"}
}

# 안전한 CSV 로딩 함수
def load_csv_safely(path):
    for enc in ['utf-8', 'cp949', 'euc-kr']:
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"인코딩 문제로 파일을 불러올 수 없습니다: {path}")

# 모든 데이터프레임 불러오기
dataframes = {name: load_csv_safely(path) for name, path in file_paths.items()}

# 격자 생성
grid_gdf = create_grid(dongdaemun_bounds)

# 각 시설 유형별로 격자 내 카운트 계산
facility_counts = {}
for name, df in dataframes.items():
    coords = coord_columns[name]
    # 각 데이터프레임의 컬럼 확인
    print(f"{name} 데이터프레임 컬럼: {df.columns.tolist()}")
    
    try:
        counts = count_facilities_in_grid(df, grid_gdf, coords['lon'], coords['lat'])
        facility_counts[name] = counts
    except Exception as e:
        print(f"{name} 처리 중 오류 발생: {e}")
        facility_counts[name] = np.zeros(len(grid_gdf))

# 결과를 격자 데이터프레임에 추가
for name, counts in facility_counts.items():
    grid_gdf[name] = counts

# 격자별 총 시설 수 계산
grid_gdf['total_facilities'] = sum(facility_counts.values())

# folium 지도 생성
center_lat = (dongdaemun_bounds['min_lat'] + dongdaemun_bounds['max_lat']) / 2
center_lon = (dongdaemun_bounds['min_lon'] + dongdaemun_bounds['max_lon']) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# 동대문구 경계 추가
folium.GeoJson(
    dongdaemun.to_json(),
    name="Dongdaemun-gu",
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'blue',
        'weight': 3,
        'fillOpacity': 0
    }
).add_to(m)

# 격자를 지도에 추가
for idx, row in grid_gdf.iterrows():
    # 격자 정보를 HTML로 구성
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px;">
        <h4>격자 정보</h4>
        <p>총 시설 수: {row['total_facilities']}</p>
        <h5>시설별 개수:</h5>
        <ul>
    """
    
    for name in facility_counts.keys():
        popup_html += f"<li>{name}: {row[name]}</li>"
    
    popup_html += "</ul></div>"
    
    # 격자 스타일 설정
    color = 'red' if row['total_facilities'] > 0 else 'lightgray'
    opacity = min(0.6, 0.1 + (row['total_facilities'] / grid_gdf['total_facilities'].max()) * 0.5) if grid_gdf['total_facilities'].max() > 0 else 0.1
    
    # 격자 추가
    folium.GeoJson(
        row.geometry.__geo_interface__,
        style_function=lambda x: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': opacity
        },
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# 범례 추가
colormap = LinearColormap(
    colors=['lightgray', 'red'],
    vmin=0,
    vmax=grid_gdf['total_facilities'].max() if grid_gdf['total_facilities'].max() > 0 else 1
)
colormap.caption = '총 시설 수'
colormap.add_to(m)

# 지도 표시
m